# **OCR Pruebas**

In [8]:
import pytesseract
from pdf2image import convert_from_path

import os

In [9]:
def extract_text(directory_path):
    pdf_texts = {}
    for filename in os.listdir(directory_path):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(directory_path, filename)
            print(f"Processing: {pdf_path}")
            pages = convert_from_path(pdf_path)
            text = ""
            for page in pages:
                text += pytesseract.image_to_string(page) + "\n"
            pdf_texts[filename] = text
    return pdf_texts

In [10]:
directory = '.\Contratos'
texts = extract_text(directory)
for filename, content in texts.items():
    print(f"--- {filename} ---")
    print(content)

Processing: .\Contratos\C01_Contrato_Prueba-01.pdf


<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Ame Contreras\AppData\Local\Temp\ipykernel_41244\2057534081.py:1: SyntaxWarning: invalid escape sequence '\C'
  directory = '.\Contratos'
C:\Users\Ame Contreras\AppData\Local\Temp\ipykernel_41244\2057534081.py:1: SyntaxWarning: invalid escape sequence '\C'
  directory = '.\Contratos'


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [2]:
import os
from PyPDF2 import PdfReader

def extract_text(directory_path):
    pdf_texts = {}
    for filename in os.listdir(directory_path):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(directory_path, filename)
            try:
                with open(pdf_path, 'rb') as file:
                    reader = PdfReader(file)
                    text = ''
                    for page in reader.pages:
                        text += page.extract_text()
                pdf_texts[filename] = text
            except Exception as e:
                print(f"Error processing {filename}: {e}")
    return pdf_texts

# Usage example:
directory = '.\\Contratos'
texts = extract_text(directory)
for filename, content in texts.items():
    print(f"--- {filename} ---")
    print(content)

unknown widths : 
[0, IndirectObject(218, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(222, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(86, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(89, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(92, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(95, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(98, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(101, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(104, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(107, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(110, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(113, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(116, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(119, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(122, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(125, 0, 1656392107072)]
unknown widths : 
[0, Indirec

--- C01_Contrato_Prueba-01.pdf ---
San Juan del Río, Querétaro Página I de 7 
CONTRATO DE ARRENDAMIENTO QUE CELEBRAN, POR UNA PARTE, EL SR. JUAN 
ALEGRIA QUINTANAR, EN LO SUCESIVO EL "ARRENDADOR", CON LA COMPARECENCIA DE LA 
SRA. MARIA; Y POR LA OTRA PARTE, "COMERCIAL IAC, .A. DE C.V.", EN LO SUCESIVO EL 
"ARRENDATARIO", REPRESENTADO EN ESTE ACTO POR SU APODERADO LEGAL EL SR 
VERTICHE, QUIENES SE SUJETAN AL TENOR DE A SIGUIENT ES: 
DECLARACIONES
1.-Del "ARRENDADOR": 
a)Ser persona fisica, dedicada al comercio, mayor de edad y con la plena capacidad jw·ídica para contratar y 
obligarse en los términos del presente contrato.
b)Los señores María y Juan, son designados por el Arrendador como beneficiarios  del pago de rentas cada 
uno al 50%; quienes se obligan a emitir los recibos fisca s correspondientes, tributan bajo el Registro 
Federal de Contribuyentes QUMJ370201FI0, con domicilio tiscal en Ray No. 22, Colonia Centro, C.P. 76800, 
San Juan del Río, Querétaro; y AERJ950629SS5 con dom

# **Updating for new docs**

In [13]:
import os
from PyPDF2 import PdfReader
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document

In [16]:
def extract_text(directory_path):
        pdf_texts = {}
        for filename in os.listdir(directory_path):
            if filename.lower().endswith('.pdf'):
                pdf_path = os.path.join(directory_path, filename)
                try:
                    with open(pdf_path, 'rb') as file:
                        reader = PdfReader(file)
                        text = ''
                        for page in reader.pages:
                            text += page.extract_text()
                    pdf_texts[filename] = text
                except Exception as e:
                    print(f"Error processing {filename}: {e}")
        return pdf_texts

In [17]:
def load_extract_split_embeddings(
    directory_path,
    persist_directory="./chroma_db",
    chunk_size=500,
    chunk_overlap=50,
    model_name="llama3.2",
    create_new=True
):
    # 1. Cargar PDFs
    loader = DirectoryLoader(
        path=directory_path,
        glob="**/*.pdf",
        loader_cls=PyPDFLoader
    )
    documentos = loader.load()

    # 2. Extraer texto (opcional si ya cargaste con PyPDFLoader, puedes omitir si prefieres)
    # Si quieres usar la función de extracción de texto manual, descomenta lo siguiente
    
    textos = extract_text(directory_path)
    docs = [{'page_content': t} for t in textos.values()]
    documentos = [Document(page_content=d['page_content'], metadata=d.get('metadata', {})) for d in docs]
    
    # 3. Dividir en chunks
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = splitter.split_documents(documentos)

    # 4. Crear embeddings
    embeddings = OllamaEmbeddings(model=model_name)

    # 5. Crear o cargar la base de datos
    if create_new:
        vectorstore = Chroma.from_documents(
            documents=chunks,
            embedding=embeddings,
            persist_directory=persist_directory
        )
    else:
        vectorstore = Chroma(
            persist_directory=persist_directory,
            embedding_function=embeddings
        )

    return vectorstore

In [20]:
# Ejemplo de uso:
vector_store = load_extract_split_embeddings(
    directory_path="./Contratos",
    create_new=True  # Cambia a False si ya tienes la base creada
)

unknown widths : 
[0, IndirectObject(218, 0, 2018116930256)]
unknown widths : 
[0, IndirectObject(222, 0, 2018116930256)]
unknown widths : 
[0, IndirectObject(86, 0, 2018116930256)]
unknown widths : 
[0, IndirectObject(89, 0, 2018116930256)]
unknown widths : 
[0, IndirectObject(92, 0, 2018116930256)]
unknown widths : 
[0, IndirectObject(95, 0, 2018116930256)]
unknown widths : 
[0, IndirectObject(98, 0, 2018116930256)]
unknown widths : 
[0, IndirectObject(101, 0, 2018116930256)]
unknown widths : 
[0, IndirectObject(104, 0, 2018116930256)]
unknown widths : 
[0, IndirectObject(107, 0, 2018116930256)]
unknown widths : 
[0, IndirectObject(110, 0, 2018116930256)]
unknown widths : 
[0, IndirectObject(113, 0, 2018116930256)]
unknown widths : 
[0, IndirectObject(116, 0, 2018116930256)]
unknown widths : 
[0, IndirectObject(119, 0, 2018116930256)]
unknown widths : 
[0, IndirectObject(122, 0, 2018116930256)]
unknown widths : 
[0, IndirectObject(125, 0, 2018116930256)]
unknown widths : 
[0, Indirec

## **Función actualizada**

In [1]:
import os
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document

In [2]:
import pytesseract
import os

# 1. Ruta al ejecutable (ajusta según tu carpeta)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# 2. Asegurar que Python vea los datos de idioma
os.environ['TESSDATA_PREFIX'] = r'C:\Program Files\Tesseract-OCR\tessdata'

In [3]:
def load_extract_split_embeddings(
    directory_path,
    persist_directory="./chroma_db",
    chunk_size=800, # Aumentado para dar más contexto legal por bloque
    chunk_overlap=100,
    model_name="llama3.2",
    create_new=True
):
    # 1. Cargar y Procesar con OCR
    # Usamos Unstructured para manejar PDFs escaneados
    all_chunks = []
    
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory_path, filename)
            print(f"Procesando con OCR: {filename}...")
            
            # El modo 'elements' extrae tablas y títulos con jerarquía
            loader = UnstructuredPDFLoader(
                file_path, 
                strategy="ocr_only", # Obligamos a OCR por ser escaneados
                languages=["spa"]    # Muy importante para las tildes y 'ñ'
            )
            docs = loader.load()
            
            # 2. División conservando Metadatos Críticos
            splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size,
                chunk_overlap=chunk_overlap
            )
            
            # Añadimos metadatos de origen para la "Sorpresa" de descarga
            for doc in docs:
                doc.metadata["source_file"] = filename
                doc.metadata["full_path"] = file_path
                
            file_chunks = splitter.split_documents(docs)
            all_chunks.extend(file_chunks)

    # 3. Embeddings e Indexación
    embeddings = OllamaEmbeddings(model=model_name)

    if create_new and os.path.exists(persist_directory):
        import shutil
        shutil.rmtree(persist_directory) # Limpiamos para evitar duplicados

    vectorstore = Chroma.from_documents(
        documents=all_chunks,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    
    print(f"Éxito: {len(all_chunks)} fragmentos indexados.")
    return vectorstore

In [4]:
# --- Configuración del Sistema de Agentes ---

llm = ChatOllama(model="llama3.2", temperature=0)
vector_store = load_extract_split_embeddings("./Contratos", create_new=True)
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

# Ajuste en el Prompt para la "Sorpresa"
template_base = """
Eres un asistente experto legal de Vertiche. 
Tu objetivo es ayudar en la toma de decisiones ágiles sobre arrendamientos[cite: 10, 14].
REGLA DE ORO: Si la información no está en el contexto, di que no la encuentras. 
Al final de cada respuesta, indica SIEMPRE el nombre del archivo fuente.

Contexto: {context}
Pregunta: {question}
"""

prompt = ChatPromptTemplate.from_template(template_base)

def format_docs(docs):
    # Aquí es donde ocurre la magia de las fuentes
    formatted = ""
    for d in docs:
        formatted += f"\n[Fuente: {d.metadata.get('source_file')}]\n{d.page_content}\n"
    return formatted

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)

Procesando con OCR: C01_Contrato_Prueba-01.pdf...
Procesando con OCR: C02_Contrato_Colima-02.pdf...
Éxito: 67 fragmentos indexados.


## Definir Herramientas

In [5]:
from langchain.tools import tool

# --- HERRAMIENTA 1: Búsqueda y Análisis (Tu Chain) ---
def tool_analizar_contrato(query: str):
    """Útil para buscar datos específicos, fechas o cláusulas dentro de los documentos."""
    return chain.invoke(query).content

# --- HERRAMIENTA 2: Filtrado y Listado ---
def tool_filtrar_datos(query: str):
    """Útil para listar contratos por ciudad o características (ej. 'Contratos en Monterrey')."""
    # Buscamos los documentos crudos para extraer metadatos
    docs = retriever.invoke(query)
    resultados = "Contratos encontrados:\n"
    for d in docs:
        resultados += f"- Archivo: {d.metadata.get('source_file')} | Ubicación: {d.metadata.get('full_path')}\n"
    return resultados

# --- HERRAMIENTA 3: Generación de Borradores ---
def tool_generar_borrador(detalles: str):
    """Útil para redactar nuevas cláusulas basadas en el estilo legal de Vertiche."""
    estilo = retriever.invoke("cláusulas estándar y formato")
    contexto_estilo = format_docs(estilo)
    
    prompt_gen = ChatPromptTemplate.from_template("""
    Eres un abogado de Vertiche. Usa este estilo: {estilo}
    Redacta lo siguiente: {detalles}
    """)
    
    # Llamada directa al LLM para creación
    chain_gen = prompt_gen | llm
    return chain_gen.invoke({"estilo": contexto_estilo, "detalles": detalles}).content

## Lista de Tools y configurar el Agente

In [8]:
from langchain_core.messages import HumanMessage, SystemMessage

# --- 1. CONFIGURACIÓN DE TOOLS (Ya las tienes, solo asegúrate de que existan) ---
tools_map = {
    "Analizar": tool_analizar_contrato,
    "Filtrar": tool_filtrar_datos,
    "Generar": tool_generar_borrador
}

# --- 2. FUNCIÓN DEL AGENTE "SMART" ---
def smart_agent(pregunta):
    print(f"Pensando en la respuesta para: {pregunta}")
    
    # El prompt le dice al modelo qué herramientas tiene
    system_prompt = f"""Eres el asistente de SmartLease AI para Vertiche. 
    Tienes acceso a estas herramientas: {list(tools_map.keys())}.
    
    Para responder, primero di qué herramienta necesitas usar. 
    Si no necesitas herramientas, responde directamente."""
    
    # 1. El modelo decide qué hacer
    respuesta_inicial = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=pregunta)
    ]).content
    
    # 2. Lógica de ejecución de herramientas
    tool_found = False
    for name in tools_map.keys():
        if name.lower() in respuesta_inicial.lower():
            print(f"Usando herramienta: {name}...")
            resultado_tool = tools_map[name](pregunta)
            
            # 3. Respuesta final combinada
            respuesta_final = llm.invoke([
                SystemMessage(content="Genera una respuesta final basada en este resultado."),
                HumanMessage(content=f"Resultado de la herramienta: {resultado_tool}")
            ]).content
            return respuesta_final
            tool_found = True
            
    if not tool_found:
        return respuesta_inicial


In [9]:
# --- 3. PRUEBA ---
resultado = smart_agent("¿Quién es el arrendador en el contrato de Centro?")
print(f"\nRESPUESTA:\n{resultado}")

Pensando en la respuesta para: ¿Quién es el arrendador en el contrato de Centro?
Usando herramienta: Analizar...

RESPUESTA:
**Análisis del Contrato**

En el contrato C01_Contrato_Prueba-01.pdf, no se menciona explícitamente el nombre del arrendador. Sin embargo, a través de la información proporcionada, podemos inferir que el arrendador es una persona o entidad que tiene derecho a recibir el alquiler.

**Representantes del Arrendador**

Se menciona en el contrato que los señores Maria y Juan son designados como beneficiarios del pago de rentas. Esto sugiere que pueden ser representantes o agente del arrendador, pero no se especifica quién es el propietario o dueño del inmueble.

**Inferencia**

En resumen, no se puede determinar con certeza quién es el arrendador en este contrato, ya que no se menciona explícitamente su identidad. Sin embargo, se puede inferir que es una persona o entidad que tiene derecho a recibir el alquiler y que Maria y Juan pueden ser representantes o agente del

In [10]:
# --- 3. PRUEBA ---
resultado = smart_agent("¿Qué se menciona en la novena cláusula del contrato C01?")
print(f"\nRESPUESTA:\n{resultado}")

Pensando en la respuesta para: ¿Qué se menciona en la novena cláusula del contrato C01?
Usando herramienta: Analizar...

RESPUESTA:
Basándome en la información proporcionada, puedo resumir los puntos clave sobre las cláusulas mencionadas:

**Quinta Cláusula**

* Se refiere al incremento de renta.
* Su aplicación es determinar el monto del pago de la renta en caso de prórroga del contrato.

**Vigésima Cláusula**

* Establece que las partes someten el contrato a los Códigos Civil y Procedimientos Civiles del Estado de Colima.
* Esto significa que estos ordenamientos legales serán aplicables al contrato en todo momento.

Espero que esta información sea útil. Si necesitas más ayuda o tienes alguna pregunta adicional, no dudes en preguntar.


In [16]:
resultado = smart_agent("¿De cuánto es el monto de renta del contrato C02_Contrato_Colima?, está en la Clausula SEGUNDA")
print(f"\nRESPUESTA:\n{resultado}")

Pensando en la respuesta para: ¿De cuánto es el monto de renta del contrato C02_Contrato_Colima?, está en la Clausula SEGUNDA
Usando herramienta: Analizar...

RESPUESTA:
Basándome en el resultado proporcionado, puedo decir que según la cláusula SEGUNDA del contrato C02_Contrato_Colima-02.pdf, el monto de renta del contrato es de $29,703.76 (VEINTINUEVE MIL SETECIENTOS TRES PESOS 76/100 M.) más el Impuesto al Valor Agregado (IVA), en términos de la Ley del Impuesto al Valor Agregado y su Reglamento, por cada mes mencionado.

En otras palabras, el monto total que debe ser pagado por cada mes es el monto de renta fijo más el IVA aplicable.


In [17]:
resultado = smart_agent("Genera una cláusula de publicidad usando el estilo de los contratos de tu base de datos")
print(f"\nRESPUESTA:\n{resultado}")

Pensando en la respuesta para: Genera una cláusula de publicidad usando el estilo de los contratos de tu base de datos
Usando herramienta: Generar...

RESPUESTA:
La respuesta final es:

**CLÁUSULA DE PUBLICIDAD**

En virtud del presente contrato, se establece que el "ARRENDADOR" podrá realizar actividades de publicidad en el inmueble arrendado, siempre y cuando se ajusten a las normas y regulaciones vigentes.

**ARTÍCULO 1: OBJETO DE LA CLÁUSULA**

La cláusula de publicidad tiene como objeto permitir al "ARRENDADOR" realizar actividades de publicidad en el inmueble arrendado, con el fin de promocionar sus productos o servicios.

**ARTÍCULO 2: CONDICIONES DE LA CLÁUSULA**

La cláusula de publicidad se regirá por las siguientes condiciones:

*   El "ARRENDADOR" podrá realizar actividades de publicidad en el inmueble arrendado, siempre y cuando no cause daño alguno al mismo ni a sus bienes.
*   Las actividades de publicidad deben ser realizadas de manera respetuosa con los demás ocupantes